<center>
<h1><b>Relatório Técnico</h1></b>
<h2>Algoritmo Quântico QAOA e o Problema do Caixeiro Viajante</b></h2>

Membros: ....
</center>

# 1 Introdução

Algoritmos Quânticos Variacionais (do inglês, *Variational Quantum Algorithms* - VQAs) tem surgido como estratégia principal para obtermos alguma vantagem computacional efetiva na era de dispositivos NISQ (**DEFINIR**). Eles podem ser utilizados em vários escopos, gerando diversas estruturas de níveis e complexidades distintas. Apesar disso, podemos destacar alguns elementos básicos que todos os VQAs tem em comum. A Figura 1.1 explicita a estrutura habitual de um VQA. Podemos separá-los inicialmente em dois blocos: uma QPU (Quantum Processing Unit) que roda um algoritmo quântico parametrizado; e uma CPU (Classical Processing Unit) que a partir de um algoritmo de otimização clássica atualizará os parâmetros utilizados na QPU.

![](./Figuras/vqa-estrutura-geral.png)

**Figura 1.1: Estrutura geral de um algoritmo quântico variacional. Fonte: Cezero et al. (2021)**

O algoritmo variacional tem como input um conjunto de dados $\{p_k\}$, uma função custo $C(\theta)$ que codifica a solução do problema, e um *Ansatz* (circuito quântico parametrizado por um conjunto discreto de ângulos $\theta$). A saída será a solução do problema, que dependerá da forma como definimos a função custo, podendo ser um estado quântico, uma distribuição de probabilidade, ou algum elemento do *Ansatz*. A função custo essencialmente mapeia os valores dos parâmetros $\theta$ em números reais, ou seja, ela define uma superfície de busca. Podemos escreve-lá da seguinte forma: 

$$
C(\theta) = \sum_k f_k (\theta,\{p_k\}) .
$$

O algoritmo clássico de otimização terá a tarefa de encontrar sob essa superfície os valores $\theta^*$ que minimizam a função custo, sendo 

$$
\theta^* = \arg\min_{\theta} \, C(\theta) ,
$$

rodando em loop, e a cada iteração utiliza um circuito quântico para estimar o valor esperado do operador hamiltoniano que define o problema, a partir de alguma estratégia a depender da forma do *Ansatz*. A solução final do problema será obtida quando algum critério de parada previamente estabelecido for atendido.

O *Quantum Approximate Optimization Algorithm* (QAOA) utiliza o *princípio variacional* da mecânica quântica
para encontrar o valor esperado do operador Hamiltoniano que define o problema, e a partir disso realizam operações para encontrar a resposta do problema. Por conseguinte, se mostra necessário investigar o princípio variacional para compreendermos a estrutura por trás do algoritmo.

# 2 Princípio Variacional

Suponha que queremos calcular o valor do estado fundamental de energia $E_f$ para um sistema descrito pelo hamiltoniano $\hat{H}$, porém não conseguimos resolver a equação de Schrödinger (por qualquer limitação que seja). O Princípio Variacional lhe dará um limite superior para $E_f$, e com algum trabalho conseguimos derivar o valor exato (**GRIFFITHS; SCHROETER, 2018**). Podemos escrever $\hat{H}$ como

$$
\hat{H} = \sum_i \lambda_i \, |\psi_i\rangle \langle \psi_i|,
$$

sendo $\lambda_i$ os autovalores e $|\psi_i\rangle$ os autovetores que formam a base para o operador $\hat{H}$. O valor esperado de $\hat{H}$ em um estado qualquer $|\Psi\rangle$ é dado por:

$$
\langle \hat{H} \rangle_{\Psi}
= \langle \Psi \mid \hat{H} \mid \Psi \rangle.
$$

Substituindo $\hat{H}$ pelas duas equações acima obtemos

$$
\begin{aligned}
\langle \hat{H} \rangle_{\Psi}
&= \langle \Psi \mid
\left(
\sum_i \lambda_i \, |\psi_i\rangle \langle \psi_i|
\right)
\mid \Psi \rangle \\
&= \sum_i \lambda_i \,
\langle \Psi \mid \psi_i \rangle
\langle \psi_i \mid \Psi \rangle \\
&\Rightarrow
\langle \hat{H} \rangle_{\Psi}
= \sum_i \lambda_i \, \lvert \langle \psi_i \mid \Psi \rangle \rvert^2 .
\end{aligned}
$$

Uma vez que todos os termos da expansão são positivos, e pelo menos um deles envolve o valor de $\lambda_{min}$, fica claro que:

$$
\lambda_{\min}
\le \langle \hat{H} \rangle_{\Psi}
= \langle \Psi \mid \hat{H} \mid \Psi \rangle
= \sum_i \lambda_i \, \lvert \langle \psi_i \mid \Psi \rangle \rvert^2
$$

e como por definição $E_f$ é igual ao menor autovalor de $\hat{H}$, encontramos um limite máximo para o mesmo. Algo importante a se notar é que o valor de energia esperado para o estado fundamental $|\Psi_f\rangle$ é:

$$
\langle \Psi_f \mid \hat{H} \mid \Psi_f \rangle
= \langle \Psi_f \mid \lambda_{\min} \mid \Psi_f \rangle
= \lambda_{\min}.
$$

Consequentemente:

$$
\langle \hat{H} \rangle_{\Psi_f}
= \lambda_{\min}.
$$

A equação acima nos diz que para encontrarmos $E_f$ devemos calcular o valor esperado do operador hamiltoniano $\hat{H}$ correspondente ao estado fundamental $|\Psi_f\rangle$.



# 3 Quantum Approximate Optimization Algorithm

O QAOA é um algoritmo variacional inicialmente projetado para resolver problemas de Otimização Combinatória. Ele é inspirado no Algoritmo de Evolução Adiabática (**FARHI et al., 2000**), porém ao contrário desse, busca apenas uma resposta
aproximada para o problema em questão. A sua função custo codifica o valor esperado de um Hamiltoniano (que aqui chamaremos de Função Objetivo) sob um estado $|\Psi\rangle$, mas nesse caso, $\Psi$ depende de um conjunto de ângulos $\{\gamma, \beta\}$. Adicionalmente, o algoritmo depende de um inteiro $p \geq 1$ e a precisão da aproximação melhora conforme $p$ aumenta. 

## 3.1 Otimização Combinatória

Problemas de Otimização Combinatória são especificados por $n$ bits e $m$ cláusulas, onde cada cláusula é uma restrição em um subconjunto de bits. Definimos uma função objetivo como o número de cláusulas satisfeitas da seguinte forma:

$$
C(z) = \sum_{\alpha=1}^{m} C_\alpha ,
$$

onde $z = z_1z_2...z_n$ é uma string e $C_{\alpha} = 1$ se $z$ satisfaz a cláusula $\alpha$ e $C_{\alpha} = 0$ caso
contrário. Algoritmos de Otimização Aproximada buscam uma string $z$ para qual $C(z)$ se aproxima do máximo (ou mínimo) de $C$.

## 3.2 Operador Gamma e Operador de Mistura

Vamos construir um operador $U$ que depende de um ângulo $\gamma$ utilizando a equação acima da seguinte forma:

$$
U(z,\gamma) = e^{-i\gamma C(z)}
= \prod_{\alpha=1}^{m} e^{-i\gamma C_\alpha} ,
$$

O operador $U(z,\gamma)$ é conhecido como *Operador Gamma*, e utilizamos esse operador para evoluir o estado inicial do problema para o estado final contendo a solução.

Porém, suponha que um dado estado inicial seja escrito como $|\psi\rangle = \sum_x \psi_x |x\rangle$ e queremos encontrar o estado fundamental de algum Hamiltoniano na forma $H_c = \sum_x E_x |x\rangle\langle x|$. Aplicando o operador $U(z,\gamma)$, tendo $H_c$ como função objetivo, obtemos:

$$
|\psi(\gamma)\rangle
= U(z,\gamma)|\psi\rangle
= \sum_x e^{-i\gamma E_x}\,\psi_x |x\rangle.
$$

Logo, a probabilidade associada à medida de $|\psi(\gamma)\rangle$ será dada por

$$
p_x
= |\langle x|\psi(\gamma)\rangle|^2
= |e^{-i\gamma E_x}\psi_x|^2
= |\psi_x|^2.
$$

Isso nos diz que a probabilidade $p_x$ de encontrarmos um estado $|x\rangle$ a partir do estado evoluído $|\psi(\gamma)\rangle$, utilizando o operador Gamma, é a mesma que obteríamos se medíssemos o estado inicial $|\psi\rangle$. O que é um problema, uma vez que queremos maximizar a função $\langle \psi(\gamma)|H_c|\psi(\gamma)\rangle,$ que, neste caso, é constante. Para resolvê-lo, vamos introduzir o *Operador de Mistura*, definido como

$$
B = \sum_{j=1}^{n} \sigma_j^x ,
$$

e sua atuação no espaço de Hilbert, com a dependência de um ângulo $\beta$, dada por:

$$
U(B,\beta) = e^{-i\beta B}
= \prod_{j=1}^{n} e^{-i\beta \sigma_j^x}.
$$

Agora o funcional de energia $\langle \psi(\gamma,\beta)|H_c|\psi(\gamma,\beta)\rangle$ não é mais constante e podemos maximizá-lo variando os ângulos $\gamma$ e $\beta$.

## 3.3 Função Custo e Estado Quântico Variacional

O estado inicial será a seguinte superposição:

$$
|s\rangle = \frac{1}{\sqrt{2^n}} \sum_z |z\rangle .
$$

Para qualquer inteiro $p \ge 1$ e $2p$ ângulos $\gamma_1,\ldots,\gamma_p = \gamma$ e $\beta_1,\ldots,\beta_p = \beta$,
vamos definir o seguinte estado parametrizado:

$$
|\gamma,\beta\rangle
= U(B,\beta_p) U(C,\gamma_p) \cdots U(B,\beta_1) U(C,\gamma_1) |s\rangle .
$$

Se $F_p$ for a função custo, definida como o valor esperado de $C(z)$ nesse estado, temos que

$$
F_p(\gamma,\beta)
= \langle \gamma,\beta | C(z) | \gamma,\beta \rangle .
$$

Adicionalmente, definimos $M_p$ como o máximo de $F_p$ sob todos os ângulos, ou seja,

$$
M_p = \max_{\gamma,\beta} F_p(\gamma,\beta).
$$

(**FARHI; GOLDSTONE; GUTMANN, 2014**) mostraram que:
$$
\lim_{p \to \infty} M_p = \max_z C(z).
$$

Esse resultado mostra que, ao criarmos um estado quântico dependente do conjunto de ângulos $(\gamma,\beta)$ e de uma constante $p$ (que evolui através da dinâmica gerada pelos operadores gamma e de mistura), se fizermos $p \to \infty$ e medirmos a função custo como o valor esperado da função objetivo sob o estado parametrizado, encontraremos uma boa aproximação para o máximo de $F_p$ (que é a solução exata do problema).

# 4 Problema do Caixeiro Viajante

O problema do Caixeiro Viajante, ou *Travelling Salesman Problem (TSP)* é um problema largamente estudado no ramo da combinatória e da teoria da complexidade computacional. Como demonstrado por (**GAREY; JOHNSON, 1979**) esse é um problema NP-Completo (*Nondeterministic Polynomial-Time Complete*), *i.e.*, dada uma solução viável, a mesma pode ser checada em tempo polinomial em uma Máquina de Turing Não-Determinística, porém encontrá-la não é uma tarefa simples. A formulação geral do problema passa a ser caracterizada pelo seguinte questionamento: *Dado um grupo de cidades e o custo para viajar entre pares do grupo, qual é o caminho que passa por todas as cidades e retorna ao ponto inicial minimizando o custo total atrelado à viagem*?

Além das aplicações óbvias voltadas a entregas, transporte e logística, (**COOK, 2015**) apresentou diversas áreas que podem se beneficiar com uma solução viável para o TSP, como o Mapeamento de Genoma, Automação de Máquinas Industriais,
ou até mesmo Testes de Microprocessadores. (**MATAI; SINGH; MITTAL, 2010**) apresentaram duas classificações distintas para o TSP, são elas: *symmetric travelling salesman problem* (sTSP) e *asymmetric travelling salesman problem* (aTSP), respectivamente Problema do Caixeiro Viajante Simétrico e Assimétrico. Neste trabalho será considerado apenas o sTSP.

## 4.1 Formulação do Problema Simétrico

Podemos formular o TSP como um problema de *Programação Linear Inteira*, em que, dada uma matriz simétrica $D = (d_{ij})$ (onde $d_{ij}$ representa a distância de $i$ para $j$), queremos encontrar um conjunto solução tal que a soma das
distâncias $d_{ij}$ de pontos consecutivos no conjunto seja mínima.

As variáveis $x_{ij}$ definem o problema caracterizado por um grafo $G = (N, A)$, em que o conjunto de vértices
$N = \{n_1, n_2, \ldots, n_k\}$ representa as cidades e o conjunto $A = \{(i,j) : i,j \in N\}$ as arestas. Assumimos $x_{ij} = 1$ se a aresta $(i,j)$ faz parte do conjunto solução, e $x_{ij} = 0$ caso contrário. O problema então é
traduzido para uma formulação que permite encontrar o *Caminho Hamiltoniano* que minimiza a respectiva função objetivo (sujeita a algumas restrições). Ou seja, temos:

**Formulação 1**

Minimizar:
$$
z = \sum_{i<j} d_{ij} x_{ij}
$$

Sujeita a:
$$
\sum_{i<k} x_{ik} + \sum_{j>k} x_{kj} = 2,
\quad \forall j \in N,
\quad k \in N
$$

$$
\sum_{i,j \in S} x_{ij} \le |S| - 1,
\quad (S \subset N)
$$

$$
x_{ij} \in \{0,1\},
\quad \forall (i,j) \in A
$$

A primeira e a terceira restrição acima são conhecidas, na Teoria dos Grafos, como *restrições de grau* e de *integralidade*, respectivamente. Elas servem para garantir que, em cada vértice do grafo, tenhamos apenas duas arestas (representando a entrada e a saída da cidade). O conjunto $S$ é especificado como um subconjunto dos vértices de um subgrafo de $G$.

A segunda restrição, por sua vez, é uma *restrição de eliminação de subtour*, que garante a não existência de caminhos pré-hamiltonianos, isto é, caminhos hamiltonianos que não incluem todos os vértices do grafo.

## 4.2 Exemplo

A título de exemplo, vejamos como utilizar a *Formulação 1* para descrever um problema com $4$ cidades. O problema é especificado na Figura 4.1, indicando que as distâncias podem ser alocadas em uma matriz $D_s$, chamada matriz de
distâncias, e definida por:

$$
D_s =
\begin{bmatrix}
0      & d_{12} & d_{13} & d_{14} \\
d_{12} & 0      & d_{23} & d_{24} \\
d_{13} & d_{23} & 0      & d_{34} \\
d_{14} & d_{24} & d_{34} & 0
\end{bmatrix}.
$$

Repare que os elementos na diagonal principal $d_{ii}$ são nulos.

![](./Figuras/tsp.png)
**Figura 4.1: sTSP para 4 cidades**

Dada a simetria do problema, existe a possibilidade do uso de variáveis restrito ao número de arestas (caminhos) do problema. Seguindo as restrições propostas, o problema pode ser descrito, neste caso, pela formulação abaixo:

**Minimizar:**
$$
z = d_{12}x_{12} + d_{13}x_{13} + d_{14}x_{14}
  + d_{23}x_{23} + d_{24}x_{24} + d_{34}x_{34}.
$$

**Sujeita a:**
$$
x_{12} + x_{13} + x_{14} = 2
$$
$$
x_{12} + x_{23} + x_{24} = 2
$$
$$
x_{13} + x_{23} + x_{34} = 2
$$
$$
x_{14} + x_{24} + x_{34} = 2
$$

$$
x_{12} + x_{13} \ge 1
$$
$$
x_{12} + x_{14} \ge 1
$$
$$
x_{13} + x_{14} \ge 1
$$
$$
x_{23} + x_{24} \ge 1
$$

$$
x_{ij} \in \{0,1\}, \quad \forall\, i,j \in N.
$$

A formulação apresentada possui um total de seis variáveis de decisão com oito restrições, sendo que quatro representam equações e a outra metade denota um sistema de inequações.

# 5 Formulação do sTSP em QUBO

Programação Quadrática Binária Irrestrita (ou *QUBO*, do inglês *Quadratic Unconstrained Binary Optimization*), é um ramo da matemática combinatória com diversas aplicações em diferentes áreas, como *Machine
Learning*, Economia e, mais relevante para o presente trabalho, Computação Quântica. Problemas do tipo QUBO são escritos como:

$$
E(x) = \sum_{i,j} x_i Q_{ij} x_j, \quad x_{i,j} \in \{0,1\},
$$

onde $x_i$ são variáveis binárias (i.e., assumem apenas valor $0$ ou $1$) e $Q_{ij}$ é uma matriz simétrica. Resolver um problema QUBO se resume a encontrar os valores das variáveis binárias que compõem o problema e que minimizam a função custo $E(x)$. Problemas do tipo QUBO são NP-hard, porém diversos avanços estão sendo feitos utilizando meta-heurísticas
para resolvê-los. Tais procedimentos são de grande valor para a mecânica quântica, como pontuado por (**GLOVER; KOCHENBERGER; YU, 2018**).

O interesse por escrever problemas conhecidos como QUBOs tem crescido recentemente na área de Computação Quântica, pelo advento de computadores quânticos baseados em *Quantum Annealing* (**JOHNSON et al., 2011**). Em linhas simples, um computador quântico construído dessa maneira utiliza o Teorema Adiabático da mecânica quântica para encontrar o valor mínimo de um Hamiltoniano na formulação de *Ising* que caracteriza o problema. 

## 5.2 Formulação do problema simétrico como um QUBO

Para demonstrarmos a formulação do sTSP como um QUBO, partiremos inicialmente do exemplo acima. Fazendo uma mudança dos índices nas variáveis de decisão, o problema pode ser reescrito como:

$$
(x_{12}, x_{13}, x_{14}, x_{23}, x_{24}, x_{34})
\;\longrightarrow\;
(x_1, x_2, x_3, x_4, x_5, x_6).
$$

**Minimizar:**
$$
z = d_1 x_1 + d_2 x_2 + d_3 x_3 + d_4 x_4 + d_5 x_5 + d_6 x_6
$$

**Sujeita a:**
$$
x_1 + x_2 + x_3 = 2
$$
$$
x_1 + x_4 + x_5 = 2
$$
$$
x_2 + x_4 + x_6 = 2
$$
$$
x_3 + x_5 + x_6 = 2
$$

$$
x_1 + x_2 \ge 1
$$
$$
x_1 + x_3 \ge 1
$$
$$
x_2 + x_3 \ge 1
$$
$$
x_4 + x_5 \ge 1
$$

$$
x_i \in \{0,1\}
$$

Agora, utilizando o procedimento de *Glover*, vamos lidar com as restrições
impostas à função custo.

- Primeiro lidamos com as igualdades utilizando:
$$
x + y = 1 \;\longrightarrow\; P(1 - x - y + 2xy)
$$

$$
x_1 + x_2 + x_3 = 2
\;\longrightarrow\;
P\left(4 - 3x_1 - 3x_2 - 3x_3
+ 2x_1x_2 + 2x_1x_3 + 2x_2x_3\right)
$$

$$
x_1 + x_4 + x_5 = 2
\;\longrightarrow\;
P\left(4 - 3x_1 - 3x_4 - 3x_5
+ 2x_1x_4 + 2x_1x_5 + 2x_4x_5\right)
$$

$$
x_2 + x_4 + x_6 = 2
\;\longrightarrow\;
P\left(4 - 3x_2 - 3x_4 - 3x_6
+ 2x_2x_4 + 2x_2x_6 + 2x_4x_6\right)
$$

$$
x_3 + x_5 + x_6 = 2
\;\longrightarrow\;
P\left(4 - 3x_3 - 3x_5 - 3x_6
+ 2x_3x_5 + 2x_3x_6 + 2x_5x_6\right)
$$

- Agora lidamos com as desigualdades utilizando:
$$
x + y \ge 1 \;\longrightarrow\; P(1 - x - y + xy)
$$

$$
x_1 + x_2 \ge 1
\;\longrightarrow\;
P(1 - x_1 - x_2 + x_1x_2)
$$

$$
x_1 + x_3 \ge 1
\;\longrightarrow\;
P(1 - x_1 - x_3 + x_1x_3)
$$

$$
x_2 + x_3 \ge 1
\;\longrightarrow\;
P(1 - x_2 - x_3 + x_2x_3)
$$

$$
x_4 + x_5 \ge 1
\;\longrightarrow\;
P(1 - x_4 - x_5 + x_4x_5)
$$

Assim, a forma irrestrita do problema se restringe em minimizar a seguinte função custo:

$$
\begin{aligned}
z =\;& d_1 x_1 + d_2 x_2 + d_3 x_3 + d_4 x_4 + d_5 x_5 + d_6 x_6 \\
&+ P\left(4 - 3x_1 - 3x_2 - 3x_3
+ 2x_1x_2 + 2x_1x_3 + 2x_2x_3\right) \\
&+ P\left(4 - 3x_1 - 3x_4 - 3x_5
+ 2x_1x_4 + 2x_1x_5 + 2x_4x_5\right) \\
&+ P\left(4 - 3x_2 - 3x_4 - 3x_6
+ 2x_2x_4 + 2x_2x_6 + 2x_4x_6\right) \\
&+ P\left(4 - 3x_3 - 3x_5 - 3x_6
+ 2x_3x_5 + 2x_3x_6 + 2x_5x_6\right) \\
&+ P\left(1 - x_1 - x_2 + x_1x_2\right)
+ P\left(1 - x_1 - x_3 + x_1x_3\right) \\
&+ P\left(1 - x_2 - x_3 + x_2x_3\right)
+ P\left(1 - x_4 - x_5 + x_4x_5\right).
\end{aligned}
$$

## 5.3 QUBO para Ising

Estamos interessados em escrever problemas do tipo QUBO na formulação de Ising, que é mais adequada para a computação quântica. A formulação geral de um QUBO considera que as variáveis $x_i$ possuem valores $x_i = 0$ ou $x_i = 1$ (*i.e.* dada a função custo $E(x)$ o valor mínimo/máximo será dado por uma configuração de variáveis binárias $x_i$ com valores de 0 ou 1). Porém queremos encontrar uma forma de reescrever $E(x)$ em termos de variáveis que representam os spins atômicos, assim como no modelo de Ising, $i.e.$, $x_i \in \{−1, 1\}$. Para fazer isto, (**LUCAS, 2014**) introduziu a seguinte mudança de variáveis:

$$
x_i \;\longrightarrow\; \frac{1 - \sigma_i^z}{2}.
$$

onde $\sigma_i^z$ é o operador de Pauli-$Z$ relacionado à variável $i$. Assim, se $x$ for a configuração correspondente ao menor valor de energia da função $E(x)$, o estado $|x\rangle$ será um autoestado do operador Hamiltoniano $H$,
correspondente ao estado fundamental de energia do sistema.

Sendo assim, podemos reescrever a equação acima em termos de produtos tensoriais de portas $\sigma^z = Z$, por exemplo,

$$
x_5 \;\longrightarrow\;
\frac{1}{2}\bigl(I \otimes I \otimes I \otimes I \otimes (I - Z) \otimes I\bigr),
\quad \text{para } N = 6.
$$

Dessa forma, conseguimos escrever o problema no formalismo de Ising. A título de exemplo, a forma explícita do Hamiltoniano de Ising que representa a codificação da função custo do sTSP pode ser encontrada abaixo:

$$
\begin{aligned}
\hat{H}_{\text{sTSP}} =\;&
\frac{d_1}{2}\bigl((I - Z) \otimes I^{\otimes 5}\bigr)
+ \frac{d_2}{2}\bigl(I \otimes (I - Z) \otimes I^{\otimes 4}\bigr)
+ \frac{d_3}{2}\bigl(I^{\otimes 2} \otimes (I - Z) \otimes I^{\otimes 3}\bigr) \\
&+ \frac{d_4}{2}\bigl(I^{\otimes 3} \otimes (I - Z) \otimes I^{\otimes 2}\bigr)
+ \frac{d_5}{2}\bigl(I^{\otimes 4} \otimes (I - Z) \otimes I\bigr)
+ \frac{d_6}{2}\bigl(I^{\otimes 5} \otimes (I - Z)\bigr) \\[6pt]
&+ P\Bigl(
-\frac{3}{2}\bigl((I - Z) \otimes I^{\otimes 5}\bigr)
-\frac{3}{2}\bigl(I \otimes (I - Z) \otimes I^{\otimes 4}\bigr)
-\frac{3}{2}\bigl(I^{\otimes 2} \otimes (I - Z) \otimes I^{\otimes 3}\bigr) \\
&\qquad
+ \frac{2}{4}\bigl((I - Z) \otimes (I - Z) \otimes I^{\otimes 4}\bigr)
+ \frac{2}{4}\bigl((I - Z) \otimes I \otimes (I - Z) \otimes I^{\otimes 3}\bigr)
+ \frac{2}{4}\bigl(I \otimes (I - Z) \otimes (I - Z) \otimes I^{\otimes 3}\bigr)
\Bigr) \\[6pt]
&+ P\Bigl(
-\frac{3}{2}\bigl(I^{\otimes 3} \otimes (I - Z) \otimes I^{\otimes 2}\bigr)
-\frac{3}{2}\bigl(I^{\otimes 4} \otimes (I - Z) \otimes I\bigr)
-\frac{3}{2}\bigl(I^{\otimes 5} \otimes (I - Z)\bigr) \\
&\qquad
+ \frac{2}{4}\bigl((I - Z) \otimes I^{\otimes 2} \otimes (I - Z) \otimes I^{\otimes 2}\bigr)
+ \frac{2}{4}\bigl((I - Z) \otimes I^{\otimes 3} \otimes (I - Z) \otimes I\bigr)
+ \frac{2}{4}\bigl(I^{\otimes 3} \otimes (I - Z) \otimes (I - Z) \otimes I\bigr)
\Bigr) \\[6pt]
&+ P\Bigl(
-\frac{3}{2}\bigl(I^{\otimes 2} \otimes (I - Z) \otimes I^{\otimes 3}\bigr)
-\frac{3}{2}\bigl(I^{\otimes 4} \otimes (I - Z) \otimes I\bigr)
-\frac{3}{2}\bigl(I^{\otimes 5} \otimes (I - Z)\bigr) \\
&\qquad
+ \frac{2}{4}\bigl(I^{\otimes 2} \otimes (I - Z) \otimes I \otimes (I - Z) \otimes I\bigr)
+ \frac{2}{4}\bigl(I^{\otimes 2} \otimes (I - Z) \otimes I^{\otimes 2} \otimes (I - Z)\bigr)
+ \frac{2}{4}\bigl(I^{\otimes 4} \otimes (I - Z) \otimes (I - Z)\bigr)
\Bigr) \\[6pt]
&+ P\Bigl(
-\frac{1}{2}\bigl((I - Z) \otimes I^{\otimes 5}\bigr)
-\frac{1}{2}\bigl(I \otimes (I - Z) \otimes I^{\otimes 4}\bigr)
+ \frac{1}{4}\bigl((I - Z) \otimes (I - Z) \otimes I^{\otimes 4}\bigr)
\Bigr) \\[6pt]
&+ P\Bigl(
-\frac{1}{2}\bigl((I - Z) \otimes I^{\otimes 5}\bigr)
-\frac{1}{2}\bigl(I^{\otimes 2} \otimes (I - Z) \otimes I^{\otimes 3}\bigr)
+ \frac{1}{4}\bigl((I - Z) \otimes I \otimes (I - Z) \otimes I^{\otimes 3}\bigr)
\Bigr) \\[6pt]
&+ P\Bigl(
-\frac{1}{2}\bigl(I \otimes (I - Z) \otimes I^{\otimes 4}\bigr)
-\frac{1}{2}\bigl(I^{\otimes 2} \otimes (I - Z) \otimes I^{\otimes 3}\bigr)
+ \frac{1}{4}\bigl(I \otimes (I - Z) \otimes (I - Z) \otimes I^{\otimes 3}\bigr)
\Bigr) \\[6pt]
&+ P\Bigl(
-\frac{1}{2}\bigl(I^{\otimes 3} \otimes (I - Z) \otimes I^{\otimes 2}\bigr)
-\frac{1}{2}\bigl(I^{\otimes 4} \otimes (I - Z) \otimes I\bigr)
+ \frac{1}{4}\bigl(I^{\otimes 3} \otimes (I - Z) \otimes (I - Z) \otimes I\bigr)
\Bigr) \\[6pt]
&+ 20P \, I^{\otimes 6}.
\end{aligned}
$$

## 6 Cenário de Implementação (Hands-On 3 e 4)

A referida atividade requer a implementação do TSP com QAOA por meio do cenário com 4 cidades.

O código do cenário pode ser acessado em [url aqui](http:urlaqui).


# Referências (ATUALIZAR!!!)

CEREZO, M. et al. Variational Quantum Algorithm. Nature Commuications, London, v.3, n.1, p.625–644, 2021.
